In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/monkeypox-skin-lesion-dataset/Monkeypox_Dataset_metadata.csv
/kaggle/input/monkeypox-skin-lesion-dataset/Fold1/Fold1/Fold1/Val/Monkeypox/M34_02_07.jpg
/kaggle/input/monkeypox-skin-lesion-dataset/Fold1/Fold1/Fold1/Val/Monkeypox/M53_01_02.jpg
/kaggle/input/monkeypox-skin-lesion-dataset/Fold1/Fold1/Fold1/Val/Monkeypox/M50_03_04.jpg
/kaggle/input/monkeypox-skin-lesion-dataset/Fold1/Fold1/Fold1/Val/Monkeypox/M34_04_07.jpg
/kaggle/input/monkeypox-skin-lesion-dataset/Fold1/Fold1/Fold1/Val/Monkeypox/M34_01_09.jpg
/kaggle/input/monkeypox-skin-lesion-dataset/Fold1/Fold1/Fold1/Val/Monkeypox/M34_03_13.jpg
/kaggle/input/monkeypox-skin-lesion-dataset/Fold1/Fold1/Fold1/Val/Monkeypox/M17_02_10.jpg
/kaggle/input/monkeypox-skin-lesion-dataset/Fold1/Fold1/Fold1/Val/Monkeypox/M50_03_06.jpg
/kaggle/input/monkeypox-skin-lesion-dataset/Fold1/Fold1/Fold1/Val/Monkeypox/M17_01_13.jpg
/kaggle/input/monkeypox-skin-lesion-dataset/Fold1/Fold1/Fold1/Val/Monkeypox/M34_02_10.jpg
/kaggle/input/monkeypox-s

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader, random_split, Subset, TensorDataset
from transformers import ViTModel
import os
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import numpy as np
import cv2

# Paths and parameters
folder_path = "/kaggle/input/monkeypox-skin-lesion-dataset/Augmented Images/Augmented Images"
subfolders = ['Monkeypox_augmented', 'Others_augmented']
image_size = (224, 224)
batch_size = 16
epochs = 30
learning_rate = 0.0001
gamma = 0.5  # Weight for combining deep and broad features

# Image transformations
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load dataset
data = []
labels = []
label_map = {subfolder: idx for idx, subfolder in enumerate(subfolders)}

for subfolder in subfolders:
    subfolder_path = os.path.join(folder_path, subfolder)
    for filename in os.listdir(subfolder_path):
        img_path = os.path.join(subfolder_path, filename)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, image_size)
            data.append(img)
            labels.append(label_map[subfolder])

data = np.array(data).astype("float32")
labels = np.array(labels)

# Create PyTorch datasets
data = torch.tensor(data).permute(0, 3, 1, 2) / 255.0  # Normalize
labels = torch.tensor(labels)

# Create a TensorDataset
dataset = TensorDataset(data, labels)

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Broad Attention Mechanism
class BroadAttention(nn.Module):
    def __init__(self, hidden_size, num_layers):
        super(BroadAttention, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.attention = nn.MultiheadAttention(embed_dim=hidden_size, num_heads=8)
        self.pool = nn.AdaptiveAvgPool1d(1)

    def forward(self, queries, keys, values):
        # Concatenate all layers' outputs
        queries = torch.cat(queries, dim=1)
        keys = torch.cat(keys, dim=1)
        values = torch.cat(values, dim=1)

        # Apply self-attention
        attention_output, _ = self.attention(queries, keys, values)

        # Pooling
        pooled_output = self.pool(attention_output.permute(0, 2, 1)).squeeze(-1)
        return pooled_output

# Define B2VIT Model
class B2VITNet(nn.Module):
    def __init__(self, vit_model_name, num_classes, gamma=0.5):
        super(B2VITNet, self).__init__()
        self.vit = ViTModel.from_pretrained(vit_model_name, output_attentions=True)
        self.hidden_size = self.vit.config.hidden_size
        self.num_layers = len(self.vit.encoder.layer)
        self.gamma = gamma

        # Broad Attention Level 1
        self.broad_attention_1 = BroadAttention(hidden_size=self.hidden_size, num_layers=self.num_layers)

        # Refinement Layer (Broad Attention Level 2)
        self.refinement_attention = nn.MultiheadAttention(embed_dim=self.hidden_size, num_heads=8)

        # Classifier
        self.classifier = nn.Sequential(
            nn.Linear(self.hidden_size, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, pixel_values):
        outputs = self.vit(pixel_values, output_hidden_states=True)
        hidden_states = outputs.hidden_states

        # Deep Features from the last CLS token
        deep_features = hidden_states[-1][:, 0, :]  # CLS token of the last layer

        # Broad Features Level 1
        queries = [layer[:, 0, :].unsqueeze(1) for layer in hidden_states]
        broad_features_1 = self.broad_attention_1(queries, queries, queries)

        # Refinement using attention
        broad_features_1 = broad_features_1.unsqueeze(1).permute(1, 0, 2)  # Prepare for MultiheadAttention
        refined_features, _ = self.refinement_attention(broad_features_1, broad_features_1, broad_features_1)
        refined_features = refined_features.permute(1, 0, 2).squeeze(1)

        # Combine Deep and Refined Broad Features
        combined_features = deep_features + self.gamma * refined_features

        # Classification
        logits = self.classifier(combined_features)
        return logits

# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs):
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        train_preds, train_targets = [], []

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * images.size(0)
            train_preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
            train_targets.extend(labels.cpu().numpy())

        train_loss /= len(train_loader.dataset)
        train_accuracy = accuracy_score(train_targets, train_preds)

        model.eval()
        val_loss = 0.0
        val_preds, val_targets = [], []

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)
                val_preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
                val_targets.extend(labels.cpu().numpy())

        val_loss /= len(val_loader.dataset)
        val_accuracy = accuracy_score(val_targets, val_preds)

        print(f"Epoch {epoch + 1}/{epochs}: Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")

# Evaluation function
def evaluate_model(model, test_loader):
    model.eval()
    test_preds, test_targets = [], []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            test_preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
            test_targets.extend(labels.cpu().numpy())

    precision = precision_score(test_targets, test_preds, average='weighted')
    recall = recall_score(test_targets, test_preds, average='weighted')
    f1 = f1_score(test_targets, test_preds, average='weighted')
    accuracy = accuracy_score(test_targets, test_preds)

    return accuracy, precision, recall, f1

# 5-Fold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold_results = []

for fold, (train_indices, val_indices) in enumerate(kf.split(dataset)):
    print(f"\nFold {fold + 1}")
    train_subset = Subset(dataset, train_indices)
    val_subset = Subset(dataset, val_indices)

    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)

    model = B2VITNet("google/vit-base-patch16-224-in21k", num_classes=2, gamma=gamma).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    train_model(model, train_loader, val_loader, criterion, optimizer, epochs)

    accuracy, precision, recall, f1 = evaluate_model(model, val_loader)
    print(f"Fold {fold + 1} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")
    fold_results.append((accuracy, precision, recall, f1))

# Average Results
average_results = np.mean(fold_results, axis=0)
print("\n5-Fold Cross-Validation Results:")
print(f"Average Accuracy: {average_results[0]:.4f}")
print(f"Average Precision: {average_results[1]:.4f}")
print(f"Average Recall: {average_results[2]:.4f}")
print(f"Average F1 Score: {average_results[3]:.4f}")



Fold 1


config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Epoch 1/30: Train Loss: 0.3734, Train Accuracy: 0.8465, Val Loss: 0.1599, Val Accuracy: 0.9343
Epoch 2/30: Train Loss: 0.1315, Train Accuracy: 0.9565, Val Loss: 0.0787, Val Accuracy: 0.9750
Epoch 3/30: Train Loss: 0.0641, Train Accuracy: 0.9839, Val Loss: 0.1010, Val Accuracy: 0.9687
Epoch 4/30: Train Loss: 0.0424, Train Accuracy: 0.9875, Val Loss: 0.0364, Val Accuracy: 0.9890
Epoch 5/30: Train Loss: 0.0200, Train Accuracy: 0.9937, Val Loss: 0.0344, Val Accuracy: 0.9922
Epoch 6/30: Train Loss: 0.0355, Train Accuracy: 0.9902, Val Loss: 0.0341, Val Accuracy: 0.9922
Epoch 8/30: Train Loss: 0.0000, Train Accuracy: 1.0000, Val Loss: 0.0254, Val Accuracy: 0.9969
Epoch 9/30: Train Loss: 0.0000, Train Accuracy: 1.0000, Val Loss: 0.0260, Val Accuracy: 0.9969
Epoch 10/30: Train Loss: 0.0000, Train Accuracy: 1.0000, Val Loss: 0.0279, Val Accuracy: 0.9969
Epoch 11/30: Train Loss: 0.0122, Train Accuracy: 0.9965, Val Loss: 0.1726, Val Accuracy: 0.9562
Epoch 12/30: Train Loss: 0.1061, Train Accuracy:

In [3]:
model_path="/kaggle/working/B2vision.pth"
torch.save(model.state_dict(),model_path)